In [260]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36'
}

def get_detail(sqlstate): 
    global _vertica_code
    global _state
    global _sqlclass
    global _desc
    global description
    _state = ''
    _vertica_code = ''
    _sqlclass = ''
    _desc = ''
    description = ''
    result = requests.get('https://www.vertica.com/docs/10.0.x/HTML/Content/Authoring/ErrorCodes/SqlState-'+sqlstate+'.htm',
                         headers = headers
                         )
    if result:
        root = etree.HTML(result.text,etree.HTMLParser())
        title_main = root.xpath("//div[@role='main']/h1/text()")
        title_top = root.xpath("//div[@id='TopicContentBody']/h1/text()")
        if title_main:
            _vertica_code = root.xpath("//div[@role='main']/pre[@class='hanging_indent']/text()")
            description = root.xpath("//div[@role='main']/p/text()")
        elif title_top:
            _vertica_code = root.xpath("//div[@id='TopicContentBody']/pre[@class='hanging_indent']/text()")
            description = root.xpath("//div[@id='TopicContentBody']/p/text()")            
        _state = sqlstate
        _sqlclass = sqlstate[0:2]
        _desc = description[1]
    return _state, _sqlclass, _desc, _vertica_code

In [261]:
headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36'
}
r = requests.get('https://www.vertica.com/docs/10.0.x/HTML/Content/Authoring/ErrorCodes/SQLStates.htm', 
                headers = headers
                )
root = etree.HTML(r.text,etree.HTMLParser())
l = []
for i in range(10):
    l.append(str(i))
toc = root.xpath('//table/tbody/tr/td/text()')
sqlcode = []
for item in toc:
    if item[0] in l:
        sqlcode.append(item)

In [262]:
with open('vertica_error_code_mapping2.csv', 'a') as f:
    header = 'sqlstate_code,sqlstate_class,sqlstate_description,vertica_code\n'
    f.write(header)
    for sqlstate in sqlcode:
        #print(sqlstate)
        state, sqlclass, desc, code= get_detail(sqlstate)
        for item in code:
            if item[0] in ('E','W'):
                i = item.split(':')
                c = i[0].split(' ')
                line = state+","+sqlclass+","+desc+","+c[1]+"\n"
                f.write(line)